In [0]:
# import section
from pyspark.sql import functions as f
from pyspark.sql import types as t
from delta.tables import DeltaTable
import logging
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('fmcg').getOrCreate()

In [0]:
%run /Workspace/apache-spark/databricks-project-fmcg-sports/utils/utilities

In [0]:
# Read product data from s3
product_bronze = (
    spark.read.format('csv')
    .option('header','true')
    .option('inferSchema', 'true')
    .load(f'{s3_bucket}/products/products.csv')
    .withColumn('readtime', f.current_timestamp())
)

#
spark.sql('DROP TABLE IF EXISTS fmcg.bronze.sb_products')
# create bronze table as it is 
product_bronze.write.mode('overwrite').saveAsTable(f'{catalog}.{bronze_schema}.sb_products')

product_bronze.show(truncate=False)

In [0]:
select(*, 'metadata').display()
# create silver')

In [0]:
product_bronze.summary().display()

In [0]:
# check if there any duplicates
duplicate_prod = product_bronze.groupby("product_id").count().filter(f.col('count') > 1 ).select("product_id")
product_bronze.filter(f.col("product_id").isin(duplicate_prod)).display()

# drop duplicate
product_silver = product_bronze.dropDuplicates()
product_silver.count()

# category name should be in cammel case
product_silver = product_silver.withColumn(
    'category', f.initcap(f.col('category'))
)

In [0]:
product_silver.display()

In [0]:
product_silver.select("category").distinct().show()